In [ ]:
import pandas as pd
import multiprocessing as mp
import numpy as np
from datetime import datetime
import os
import errno

Specify Test , Output, Files

In [ ]:
test = ["D:\homework2\hw2-rdf-2016_test\hw2-rdf-2016_test.txt"]
files = []
for i in range(1,2):
    files.append("D:\homework2\data\hw2-rdf-2016_1\hw2-rdf-2016_{}.dat".format(str(i)))
output_file = "D:\homework2\output\output_final.csv"    

Function that does the work

In [ ]:
def dothejob(input):  
    for f in input:
        with open(f,'r') as f:
            print("Now working on file {} ".format(str(f).split("'")[1]))
            reader = pd.read_table(f, header=None,chunksize=50000) 
            for read_chunk in reader:    # start reading in chunks
                read_chunk=read_chunk[0].str.split(" ", expand=True)
                read_chunk['EDGES'] = read_chunk.stack().str.contains(r'^\"|<|_:').astype(np.uint8).sum(level=0)-1
                read_chunk[read_chunk['EDGES'] >= 3] = 3
                read_chunk[read_chunk['EDGES'] <=2] =2 
                read_chunk.rename(columns={0:'SUBJECT'}, inplace=True)
                #append the so-far read chunk into df_temp , which later will be writter to csv
                df_temp.append(read_chunk[['SUBJECT','EDGES']], ignore_index=True)
                #df_temp = df_temp.groupby('SUBJECT', as_index=False).sum()
                del read_chunk   ### delete the read chunk and iterate loop for next 50000 lines
            writetofile(df_temp)   #### write to csv
                
def writetofile(somedf):
    #if not os.path.exists(output_file):
    #    open(output_file, 'w').close() 
    with open(output_file,'a+') as final:
        df_temp.to_csv(final)

        
#### create an empty dataframe
df_temp = pd.DataFrame()
df_temp = df_temp.fillna(0) # with 0s rather than NaNs
#### created a temp dataframe

#  RUN
t1 = datetime.now()
dothejob(files)    # WILL RUN FOR FIRST FILE ONLY
print("finished after {} ".format(datetime.now() - t1))